In [1]:
import pandas as pd
import numpy as np

from agg import prep_demand_features
from demand_features import X_cols

pd.options.display.max_rows = 150
pd.options.display.max_columns = 250
pd.options.display.max_colwidth = None

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [2]:
h1_stats = pd.read_pickle("../data/h1_stats.pick")

In [3]:
h1_stats.describe()

,RoomsOTB,RevOTB,CxlForecast,TRN_RoomsOTB,TRN_RevOTB,TRN_CxlForecast,DaysUntilArrival,ADR_OTB,SellingPrice,RemSupply,Mon,Sat,Sun,Thu,Tue,Wed,NONTRN_RoomsOTB,NONTRN_RevOTB,NONTRN_ADR_OTB,NONTRN_CxlForecast,ACTUAL_RoomsSold,ACTUAL_ADR,ACTUAL_RoomRev,ACTUAL_NumCancels,TRN_ADR_OTB,TM30_RoomsPickup,TM30_RevPickup,TM30_ADR_Pickup,TM30_TRN_RoomsPickup,TM30_TRN_RevPickup,TM30_TRN_ADR_Pickup,TM30_NONTRN_RoomsPickup,TM30_NONTRN_RevPickup,TM30_NONTRN_ADR_Pickup,TM15_RoomsPickup,TM15_RevPickup,TM15_ADR_Pickup,TM15_TRN_RoomsPickup,TM15_TRN_RevPickup,TM15_TRN_ADR_Pickup,TM15_NONTRN_RoomsPickup,TM15_NONTRN_RevPickup,TM15_NONTRN_ADR_Pickup,TM05_RoomsPickup,TM05_RevPickup,TM05_ADR_Pickup,TM05_TRN_RoomsPickup,TM05_TRN_RevPickup,TM05_TRN_ADR_Pickup,TM05_NONTRN_RoomsPickup,TM05_NONTRN_RevPickup,TM05_NONTRN_ADR_Pickup,RoomsGapToLYA,RevGapToLYA,ADR_GapToLYA,TRN_RoomsGapToLYA,TRN_RevGapToLYA,TRN_ADR_GapToLYA,NONTRN_RoomsGapToLYA,NONTRN_RevGapToLYA,NONTRN_ADR_GapToLYA,DaysUntilArrival_STLY,RoomsOTB_Pace,ADR_OTB_Pace,RevOTB_Pace,CxlForecast_Pace,RemSupply_Pace,SellingPrice_Pace,TRN_RoomsOTB_Pace,TRN_ADR_OTB_Pace,TRN_RevOTB_Pace,TRN_CxlForecast_Pace,NONTRN_RoomsOTB_Pace,NONTRN_ADR_OTB_Pace,NONTRN_RevOTB_Pace,NONTRN_CxlForecast_Pace,TM30_RoomsPickup_Pace,TM30_ADR_Pickup_Pace,TM30_RevPickup_Pace,TM30_TRN_RoomsPickup_Pace,TM30_TRN_ADR_Pickup_Pace,TM30_TRN_RevPickup_Pace,TM30_NONTRN_RoomsPickup_Pace,TM30_NONTRN_ADR_Pickup_Pace,TM30_NONTRN_RevPickup_Pace,TM15_RoomsPickup_Pace,TM15_ADR_Pickup_Pace,TM15_RevPickup_Pace,TM15_TRN_RoomsPickup_Pace,TM15_TRN_ADR_Pickup_Pace,TM15_TRN_RevPickup_Pace,TM15_NONTRN_RoomsPickup_Pace,TM15_NONTRN_ADR_Pickup_Pace,TM15_NONTRN_RevPickup_Pace,TM05_RoomsPickup_Pace,TM05_ADR_Pickup_Pace,TM05_RevPickup_Pace,TM05_TRN_RoomsPickup_Pace,TM05_TRN_ADR_Pickup_Pace,TM05_TRN_RevPickup_Pace,TM05_NONTRN_RoomsPickup_Pace,TM05_NONTRN_ADR_Pickup_Pace,TM05_NONTRN_RevPickup_Pace
count,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.00000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.00000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.000000,12208.00000,12208.000000,12208.000000
mean,139.693562,14401.099538,18.410550,89.930374,10275.610964,16.057585,15.276540,94.521588,101.740257,65.716989,0.142857,0.142038,0.142448,0.144004,0.143267,0.143676,49.763188,4125.488574,82.780837,2.352965,156.167595,94.948845,15719.184259,77.098050,100.618447,9.912516,939.082892,0.918850,7.298984,703.86121,0.953703,2.613532,235.221682,1.081473,0.781045,17.742694,-0.264474,0.757700,36.606579,-0.334466,0.023345,-18.863885,-0.359994,-7.672592,-606.246786,-0.258184,-5.445773,-453.559906,-0.251895,-2.226818,-152.686880,-0.395004,4.550704,-1294.865487,-11.824486,10.010321,-469.331316,-14.292563,-5.459617,-825.534172,2.468077,15.276540,12.729112,12.460346,2544.981504,3.797919,-8.931193,15.078701,5.844856,14.970347,1603.077616,4.595675,6.884256,11.142054,941.90

In [5]:
mask = (h1_stats["StayDate"] < '2017-08-01')
test_mask = (h1_stats['AsOfDate'] == '2017-08-01')
h1_train = h1_stats[mask].copy()
h1_test = h1_stats[test_mask].copy()

X1_train = h1_train[X_cols]
X1_test = h1_test[X_cols]
y1_train = h1_train['ACTUAL_RoomsSold']
y1_test = h1_test['ACTUAL_RoomsSold']

In [ ]:
h1_test['id']

In [9]:
lm = LinearRegression()
lr_model = lm.fit(X1_train, y1_train)
scores = cross_val_score(lm, X1_train, y1_train, scoring='r2', cv=5)
scores.mean()

0.4740146510925262

In [11]:
rfm = RandomForestRegressor(n_jobs=-1, random_state=20)
rf_model = rfm.fit(X1_train, y1_train)
scores = cross_val_score(rfm, X1_train, y1_train, scoring='r2', cv=5)
scores.mean()

0.5131041550793125

In [14]:
xgbm = XGBRegressor(n_jobs=-1, random_state=20)
scores = cross_val_score(xgbm, X1_train, y1_train, scoring='r2', cv=5)
scores.mean()

# score without 

0.5829845385400213

In [ ]:
%%time
param_grid = {
    'learning_rate':[0.10, 0.15, 0.2],
    'n_estimators':[3000, 4000, 5000],
    'max_depth':[2, 3, 4, 5]
}

xgb_gs = GridSearchCV(estimator=xgbm, param_grid=param_grid, verbose=10)
xgb_gsm = xgb_gs.fit(X1_train, y1_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits
[CV 1/5; 1/36] START learning_rate=0.1, max_depth=2, n_estimators=3000..........
[CV 1/5; 1/36] END learning_rate=0.1, max_depth=2, n_estimators=3000; total time=   8.1s
[CV 2/5; 1/36] START learning_rate=0.1, max_depth=2, n_estimators=3000..........
[CV 2/5; 1/36] END learning_rate=0.1, max_depth=2, n_estimators=3000; total time=   8.0s
[CV 3/5; 1/36] START learning_rate=0.1, max_depth=2, n_estimators=3000..........
[CV 3/5; 1/36] END learning_rate=0.1, max_depth=2, n_estimators=3000; total time=   7.9s
[CV 4/5; 1/36] START learning_rate=0.1, max_depth=2, n_estimators=3000..........


In [ ]:
xgb_gsm.best_estimator_

In [ ]:
xgb_gsm.best_score

In [ ]:
xgb_gsm.best_params_

In [ ]:
xgb_gsm.score(X1_test, y1_test)

In [7]:
xgbm = XGBRegressor(n_jobs=-1, random_state=20, learning_rate=0.1, max_depth=2, n_estimators=4000)
xgb_model = xgbm.fit(X1_train, y1_train)

In [8]:
xgb_model.score(X1_test, y1_test)

0.46672832554432764

In [10]:
lr_model.score(X1_test, y1_test)

0.03966045063543522

In [13]:
rf_model.score(X1_test, y1_test)

0.3780098792304456